In [1]:
# running a child process with subprocess
import subprocess
proc = subprocess.Popen(['echo', 'Hello from the child!'], stdout=subprocess.PIPE)
out, err = proc.communicate()
print(out.decode('utf-8'))

Hello from the child!



In [2]:
# run independently from their parent process 
import time

proc = subprocess.Popen(['sleep', '0.1'])
while proc.poll() is None:
    print('Working...')
    time.sleep(0.1)

print('Exit status', proc.poll())

Working...
Working...
Exit status 0


In [3]:
# run many child processes in parallel
def run_sleep(period):
    proc = subprocess.Popen(['sleep', str(period)])
    return proc

start = time.time()
procs = []
for _ in range(10):
    proc = run_sleep(0.1)
    procs.append(proc)
    
# wait for them to finish their I/O and terminate with the communicate method
for proc in procs:
    proc.communicate()
end = time.time()
print('Finished in %.3f seconds' % (end - start))

Finished in 0.118 seconds


In [4]:
# pipe data from python program into a subprocess
import os
def run_openssl(data):
    proc = subprocess.Popen(
        ['openssl', 'rand', '-base64', '128'],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE)
    proc.stdin.write(data)
    proc.stdin.flush()
    return proc

In [6]:
procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    procs.append(proc)
    
for proc in procs:
    out, err = proc.communicate()
    print(out[:64])

b'B10C0eB4RSyyLv9yoTbVovWH+mhIOF9R8lpiCKrJBJ9OREHsuXUxOWIGmEVT+ART'
b'J/wswCg/GdNdoSwwuLBq7Aq/k6rlfmsAbc0pe1JOJdYDGKSW7lFqLU5G9lX2cIvp'
b'X8WEXPqzS1iJAGTcX+cb5WiQb4YylY+hBKtX17ElOHVEEKzzFY74Atfx1d8PgYxP'


In [7]:
def run_md5(input_stdin):
    proc = subprocess.Popen(
        ['md5sum'],
        stdin=input_stdin,
        stdout=subprocess.PIPE)
    return proc

In [8]:
# chains of parallel process
input_procs = []
hash_procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    input_procs.append(proc)
    hash_proc = run_md5(proc.stdout)
    hash_procs.append(hash_proc)

In [9]:
for proc in input_procs:
    proc.communicate()
for proc in hash_procs:
    out, err = proc.communicate()
    print(out.strip())

b'984f581aaf3c695afd760c33d8db1da1  -'
b'0ada0818ffa81150755480dc9310b753  -'
b'2f7b71a6ac6c52bdd8b23f0ce2a4ae75  -'


In [10]:
# timeout parameter to kill subprocess
proc = run_sleep(10)
try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()

print('Exit status', proc.poll())

Exit status -15
